In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import json
import pandas as pd
import numpy as np
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_stat_map
from nilearn.reporting import get_clusters_table
sys.path.append('..')
from utils.data import load_participant_list, get_betamap_paths

# Parameters

In [3]:
run = 'test'
first_level_dir = '/home/ubuntu/data/learning-habits/bids_dataset/derivatives/nilearn/rsa_20250218'

# First level parameters

In [4]:
# Find the first JSON file
json_file_path = glob.glob(os.path.join(first_level_dir, '**', '*_params.json'), recursive=True)[0]

# Load and print the JSON file contents
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
    for key, value in json_data.items():
        print(f"{key}: {value}")


model_name: null_durations
tr: 2.33384
hrf_model: spm
noise_model: ar1
smoothing_fwhm: 5
high_pass: 0.01
motion_type: basic
fd_thresh: 0.5
std_dvars_thresh: 2
scrub: dummies
include_physio: True
brain_mask: True
duration: iti_only
exclusion_threshold: 0.2


# Load First level data

In [8]:
base_dir = '/home/ubuntu/data/learning-habits'
sub_ids = load_participant_list(base_dir)

In [9]:
stimuli = [1, 2, 3, 4, 5, 6, 7, 8]

In [12]:
for stim in stimuli:
    betamap_paths, found_subs = get_betamap_paths(sub_ids, first_level_dir, run, f"sub-{{sub_id}}_run-{{run}}_betamap_first_stim_presentation_{stim}.nii.gz")
    print(f"Found {len(betamap_paths)} beta maps for stimulus {stim}")

Found 57 beta maps for stimulus 1
Found 57 beta maps for stimulus 2
Found 57 beta maps for stimulus 3
Found 57 beta maps for stimulus 4
Found 57 beta maps for stimulus 5
Found 57 beta maps for stimulus 6
Found 57 beta maps for stimulus 7
Found 57 beta maps for stimulus 8


In [ ]:
contrast_modulator_RL_imgs = [image.load_img(path) for path in contrast_modulator_RL_paths]
contrast_modulator_CK_imgs = [image.load_img(path) for path in contrast_modulator_CK_paths]

In [ ]:
print(f"Found {len(contrast_modulator_RL_imgs)} RL and {len(contrast_modulator_CK_imgs)} CK betamaps")

Found 57 RL and 57 CK betamaps


In [ ]:
import os
import numpy as np
import nibabel as nib
import scipy.spatial.distance as sp_dist
import matplotlib.pyplot as plt
import seaborn as sns

# Define parameters
num_subjects = 50
num_stimuli = 8
data_path = "path_to_your_betamaps"  # Update with actual path

# Initialize dictionary to store RDMs
rdms = {}

# Iterate over subjects
for subj in range(1, num_subjects + 1):
    subject_id = f"sub-{subj:02d}"  # Adjust formatting based on file naming
    betamaps = []
    
    # Load betamaps for each stimulus
    for stim in range(1, num_stimuli + 1):
        file_path = os.path.join(data_path, subject_id, f"stim-{stim:02d}_betamap.nii.gz")  # Adjust naming
        img = nib.load(file_path)
        data = img.get_fdata()
        betamaps.append(data.flatten())  # Flatten into a vector
    
    # Compute pairwise distances (e.g., correlation distance)
    rdm = sp_dist.pdist(np.array(betamaps), metric='correlation')
    rdm = sp_dist.squareform(rdm)  # Convert to square matrix
    
    rdms[subject_id] = rdm

# Plot an example RDM
example_subject = "sub-01"  # Change as needed
plt.figure(figsize=(8, 6))
sns.heatmap(rdms[example_subject], annot=False, cmap="viridis", square=True)
plt.title(f"RDM for {example_subject}")
plt.xlabel("Stimulus")
plt.ylabel("Stimulus")
plt.show()